<div align='center'>

# OVERVIEW
</div>

This notebook aims to clarify **whether or not any of the suggested Data Retrieval & Reporting services are acceptable for use for our Customer Support Center system**, as defined in the [`customer_support_rep_persona_01.md` persona document](../../customer_rep_persona_store/customer_support_rep_persona_01.md).

The suggested services are listed below:

1. **Metabase** (Business Intelligence and Reporting Platform)
2. **Grafana** (Metrics Visualization and Monitoring)

The criteria for qualifying a suggestion as a tool is defined below:

1. Python-based.
2. Open-source (free, avoid freemium as much as possible) customer-facing **and** producer-facing access.
3. Rate limit restrictions allow for appropriate edge-case testing.
4. Supports core architectural choice for LangChain ecosystem (e.g. supports asynchronous backends, etc).
5. I can currently access the tool as a human (i.e. confirmation that it is operable as a human, and has not been deprecated/restricted discreetly).
6. I can perform its expected basic operations.

## Pre-Notebook Initialization

1. Install Metabase via Docker: `docker run -d -p 3000:3000 --name metabase metabase/metabase`
2. Install Grafana via Docker: `docker run -d -p 3001:3000 --name grafana grafana/grafana-oss`
3. Configure data sources (PostgreSQL, etc.) in both platforms.
4. Create API keys/tokens for programmatic access.

<u>Notes</u>:
- Metabase provides self-service business intelligence for querying and visualizing data.
- Grafana excels at time-series visualization and real-time monitoring dashboards.

# Required Modules


In [ ]:
import os
import requests
import json
from dotenv import load_dotenv
load_dotenv()
from datetime import datetime, timedelta


## Metabase


<ol>
<li> Python-based: <b>Available</b> (via REST API)
<li> Open-source: <b>Free</b> (AGPL License for self-hosted)

<li> Maximum Rate Limit Restriction:
<ul>
<li> Self-hosted: <b>No rate limits</b> (depends on infrastructure).
<li> API endpoints have configurable rate limiting.
</ul>

<li> <b>Supports Core Architectural Choice</b> (REST API compatible with async HTTP clients)
<li> Service is confirmed to <b>still be active & accessible</b>
</ol>

### Expected Basic Operations

1. [Authenticate / Get Session Token](https://www.metabase.com/docs/latest/api#post-apisession).
2. [List Databases](https://www.metabase.com/docs/latest/api/database#get-apidatabase).
3. [Execute Query](https://www.metabase.com/docs/latest/api/dataset#post-apidataset).
4. [Get Card/Question Results](https://www.metabase.com/docs/latest/api/card#get-apicardidquery).
5. [List Dashboards](https://www.metabase.com/docs/latest/api/dashboard#get-apidashboard).
6. [Export Query Results](https://www.metabase.com/docs/latest/api/dataset#post-apidatasetformat).

**<u>Notes:</u>**
1. Metabase supports multiple database backends (PostgreSQL, MySQL, MongoDB, etc.).
2. Questions (saved queries) can be embedded in external applications.
3. Supports scheduled reports and email delivery.


### Setup

Metabase provides a REST API for programmatic access to reports and data.


In [ ]:
# Class Definition for Metabase API calls

class Simple_Metabase_Client():
    
    def __init__(
        self,
        base_url: str = None,
        username: str = None,
        password: str = None
    ):
        self.base_url = base_url or os.getenv("METABASE_URL", "http://localhost:3000")
        self.username = username or os.getenv("METABASE_USERNAME")
        self.password = password or os.getenv("METABASE_PASSWORD")
        self.SUCCESS_MSG = "[INFO]: Request performed successfully!"
        self.ERROR_MSG = "[WARNING]:"
        self.session_token = None
        
    def authenticate(self):
        """Authenticate and get session token"""
        try:
            response = requests.post(
                f"{self.base_url}/api/session",
                json={
                    "username": self.username,
                    "password": self.password
                }
            )
            response.raise_for_status()
            self.session_token = response.json().get("id")
            print(self.SUCCESS_MSG, "Authenticated successfully.")
            return True
        except Exception as e:
            print(self.ERROR_MSG, e)
            return False
    
    def _headers(self):
        """Get headers with session token"""
        return {"X-Metabase-Session": self.session_token}
    
    def list_databases(self):
        """List all configured databases"""
        try:
            response = requests.get(
                f"{self.base_url}/api/database",
                headers=self._headers()
            )
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def execute_query(self, database_id: int, query: str):
        """Execute a native SQL query"""
        try:
            response = requests.post(
                f"{self.base_url}/api/dataset",
                headers=self._headers(),
                json={
                    "database": database_id,
                    "type": "native",
                    "native": {
                        "query": query
                    }
                }
            )
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def list_dashboards(self):
        """List all dashboards"""
        try:
            response = requests.get(
                f"{self.base_url}/api/dashboard",
                headers=self._headers()
            )
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def get_card_results(self, card_id: int):
        """Get results for a saved question/card"""
        try:
            response = requests.post(
                f"{self.base_url}/api/card/{card_id}/query",
                headers=self._headers()
            )
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def export_query_csv(self, database_id: int, query: str):
        """Export query results as CSV"""
        try:
            response = requests.post(
                f"{self.base_url}/api/dataset/csv",
                headers=self._headers(),
                json={
                    "database": database_id,
                    "type": "native",
                    "native": {
                        "query": query
                    }
                }
            )
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.text
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def list_cards(self):
        """List all saved questions/cards"""
        try:
            response = requests.get(
                f"{self.base_url}/api/card",
                headers=self._headers()
            )
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None


### 1. Authenticate

Authenticates with Metabase and obtains a session token.


In [ ]:
# Metabase Client Init

metabase_client = Simple_Metabase_Client()

# Authenticating
metabase_client.authenticate()


### 2. List Databases

Lists all configured database connections.


In [ ]:
# Listing available databases

databases = metabase_client.list_databases()
if databases:
    print("Available Databases:")
    for db in databases.get('data', []):
        print(f"  - ID: {db.get('id')}, Name: {db.get('name')}")


### 3. Execute Query

Executes a SQL query against a database.


In [ ]:
# Executing a query

database_id = 1  # Adjust based on available databases
query = "SELECT * FROM accounts LIMIT 10"

results = metabase_client.execute_query(database_id, query)
if results:
    print(f"Query returned {len(results.get('data', {}).get('rows', []))} rows")


### 4. List Dashboards

Lists all available dashboards.


In [ ]:
# Listing dashboards

dashboards = metabase_client.list_dashboards()
if dashboards:
    print("Available Dashboards:")
    for dashboard in dashboards:
        print(f"  - ID: {dashboard.get('id')}, Name: {dashboard.get('name')}")


### 5. List Saved Questions

Lists all saved questions/cards.


In [ ]:
# Listing saved questions/cards

cards = metabase_client.list_cards()
if cards:
    print("Saved Questions:")
    for card in cards[:10]:  # Show first 10
        print(f"  - ID: {card.get('id')}, Name: {card.get('name')}")


### 6. Export Query to CSV

Exports query results to CSV format.


In [ ]:
# Exporting query to CSV

csv_data = metabase_client.export_query_csv(database_id, query)
if csv_data:
    print("CSV Export Preview:")
    print(csv_data[:500] if len(csv_data) > 500 else csv_data)


## Grafana


<ol>
<li> Python-based: <b>Available</b> (via REST API)
<li> Open-source: <b>Free</b> (AGPL v3 License for OSS version)

<li> Maximum Rate Limit Restriction:
<ul>
<li> Self-hosted: <b>No rate limits</b> (depends on infrastructure).
<li> API endpoints configurable with rate limiting plugins.
</ul>

<li> <b>Supports Core Architectural Choice</b> (REST API compatible with async HTTP clients)
<li> Service is confirmed to <b>still be active & accessible</b>
</ol>

### Expected Basic Operations

1. [Authenticate / API Key](https://grafana.com/docs/grafana/latest/developers/http_api/auth/).
2. [List Dashboards](https://grafana.com/docs/grafana/latest/developers/http_api/dashboard/#search-dashboards).
3. [Get Dashboard by UID](https://grafana.com/docs/grafana/latest/developers/http_api/dashboard/#get-dashboard-by-uid).
4. [List Data Sources](https://grafana.com/docs/grafana/latest/developers/http_api/data_source/).
5. [Query Data Source](https://grafana.com/docs/grafana/latest/developers/http_api/ds/).
6. [Create Alert Rule](https://grafana.com/docs/grafana/latest/developers/http_api/alerting/).

**<u>Notes:</u>**
1. Grafana excels at time-series data visualization and real-time monitoring.
2. Supports extensive plugin ecosystem for data sources (Prometheus, InfluxDB, PostgreSQL, etc.).
3. Alerting capabilities for automated notifications on metric thresholds.


### Setup

Grafana provides a REST API for programmatic access to dashboards and data sources.


In [ ]:
# Class Definition for Grafana API calls

class Simple_Grafana_Client():
    
    def __init__(
        self,
        base_url: str = None,
        api_key: str = None
    ):
        self.base_url = base_url or os.getenv("GRAFANA_URL", "http://localhost:3001")
        self.api_key = api_key or os.getenv("GRAFANA_API_KEY")
        self.SUCCESS_MSG = "[INFO]: Request performed successfully!"
        self.ERROR_MSG = "[WARNING]:"
        
    def _headers(self):
        """Get headers with API key"""
        return {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
    
    def health_check(self):
        """Check Grafana server health"""
        try:
            response = requests.get(f"{self.base_url}/api/health")
            response.raise_for_status()
            print(self.SUCCESS_MSG, "Grafana is healthy.")
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def list_dashboards(self, query: str = ""):
        """Search for dashboards"""
        try:
            response = requests.get(
                f"{self.base_url}/api/search",
                headers=self._headers(),
                params={"query": query, "type": "dash-db"}
            )
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def get_dashboard(self, uid: str):
        """Get dashboard by UID"""
        try:
            response = requests.get(
                f"{self.base_url}/api/dashboards/uid/{uid}",
                headers=self._headers()
            )
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def list_datasources(self):
        """List all data sources"""
        try:
            response = requests.get(
                f"{self.base_url}/api/datasources",
                headers=self._headers()
            )
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def query_datasource(self, datasource_uid: str, query: dict):
        """Query a data source"""
        try:
            response = requests.post(
                f"{self.base_url}/api/ds/query",
                headers=self._headers(),
                json={
                    "queries": [
                        {
                            "datasourceId": datasource_uid,
                            **query
                        }
                    ],
                    "from": "now-1h",
                    "to": "now"
                }
            )
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def list_alerts(self):
        """List all alert rules"""
        try:
            response = requests.get(
                f"{self.base_url}/api/v1/provisioning/alert-rules",
                headers=self._headers()
            )
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def get_org_info(self):
        """Get current organization info"""
        try:
            response = requests.get(
                f"{self.base_url}/api/org",
                headers=self._headers()
            )
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None


### 1. Health Check

Checks Grafana server health status.


In [ ]:
# Grafana Client Init

grafana_client = Simple_Grafana_Client()

# Checking health
grafana_client.health_check()


### 2. List Dashboards

Lists all available dashboards.


In [ ]:
# Listing dashboards

dashboards = grafana_client.list_dashboards()
if dashboards:
    print("Available Dashboards:")
    for dashboard in dashboards:
        print(f"  - UID: {dashboard.get('uid')}, Title: {dashboard.get('title')}")


### 3. List Data Sources

Lists all configured data sources.


In [ ]:
# Listing data sources

datasources = grafana_client.list_datasources()
if datasources:
    print("Configured Data Sources:")
    for ds in datasources:
        print(f"  - ID: {ds.get('id')}, Name: {ds.get('name')}, Type: {ds.get('type')}")


### 4. Get Organization Info

Retrieves current organization information.


In [ ]:
# Getting organization info

org_info = grafana_client.get_org_info()
if org_info:
    print(f"Organization: {org_info}")


### 5. List Alert Rules

Lists all configured alert rules.


In [ ]:
# Listing alert rules

alerts = grafana_client.list_alerts()
if alerts:
    print("Alert Rules:")
    for alert in alerts:
        print(f"  - {alert.get('title', alert)}")
